In [2]:
import time
import uuid
import boto3
import sagemaker

sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [3]:
model_training_s3_uri = 's3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-100GB/'

In [4]:
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost


instance_type = "ml.m5.24xlarge"
output_content_type = "parquet"

train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=model_training_s3_uri,
    content_type=train_content_type,
    distribution='ShardedByS3Key',
#    input_mode='FastFile'
)


In [5]:
hyperparameters = {
    "eta": "0.2",
    "gamma": "4",
    "max_depth": "5",
    "min_child_weight": "6",
    "num_round": "50",
    "objective": "reg:squarederror",
    "subsample": "0.7",
#    "verbosity": "2",
    "content_type":"parquet",
}


In [ ]:
from sagemaker.pytorch import PyTorch

# PyTorch Image is used to enable distributed training
estimator = PyTorch(
    source_dir="code",
    entry_point="xgboost-ray-nyctaxi-parquet.py",
    role=role,
    subnets=None,
    security_group_ids=None,
    instance_count=2, 
    instance_type="ml.m5.24xlarge",
    framework_version="1.13",
    hyperparameters=hyperparameters,
    py_version="py39"
)
estimator.fit({"train": train_input})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-03-14-06-21-42-150


2023-03-14 06:21:42 Starting - Starting the training job.